In [1]:
print("notebook started..")
!pip install -q -U transformers --no-index --find-links /kaggle/input/hf-libraries/transformers
!pip install -q -U accelerate --no-index --find-links /kaggle/input/hf-libraries/accelerate
!pip install -q -U bitsandbytes --no-index --find-links /kaggle/input/hf-libraries/bitsandbytes
print("pip installs done!")

import numpy as np
import pandas as pd
import random
import math

from collections import OrderedDict
from math import exp

import torch.nn.functional as F

from tqdm import tqdm

notebook started..
pip installs done!


# Reload prior submission.csv and optimize from that!!!
* Set `reuse_last_submission = False` to start fresh
* Specify any samples we don't think need further optimization
* Specify any samples we want to fully shuffle between optimization rounds (stuck at local optimum)

In [2]:
# Set to False to do a "clean" run
reuse_last_submission = True

if reuse_last_submission:
    samples = pd.read_csv("/kaggle/input/santa-best-code/submission.csv")    
else:
    samples = pd.read_csv("/kaggle/input/santa-2024/sample_submission.csv")

# If we think some items are well-optimized - including them here will skip them
skip = [0,1,2,4,5]

# If some samples are badly stuck at local optimum - we fully shuffle between rounds
shuffle_between_cycles = [3]

# Sorting sample 6 -[StopWords First]
* Borrowing some help from...
  
## https://www.kaggle.com/code/asalhi/sorting-sample-6-stopwords-first

In [3]:
#from nltk.corpus import stopwords
#import nltk
#import numpy as np

# Download stopwords if needed
#nltk.download('stopwords')
#stop_words = set(stopwords.words('english'))

#def custom_sort(text):
#    words = text.split()
#    stop_words_in_text = sorted([word for word in words if word.lower() in stop_words])
#    other_words = sorted([word for word in words if word.lower() not in stop_words])
#    return " ".join(stop_words_in_text + other_words)

In [4]:
# Apply the sorting to sample 2 (index 1)
#samples.loc[1, "text"] = custom_sort(samples.loc[1, "text"])
samples.loc[1,"text"] = "reindeer sleep walk the night and drive mistletoe scrooge laugh gingerbread bake chimney elf jump ornament give family advent fireplace"
#"reindeer mistletoe ornament gingerbread bake the night walk and sleep scrooge drive fireplace chimney jump elf laugh give family advent"
#"advent and bake chimney drive elf family fireplace gingerbread give jump laugh mistletoe night ornament reindeer scrooge sleep the walk"
#"walk the reindeer ornament scrooge sleep night mistletoe laugh jump give gingerbread fireplace family elf drive chimney bake and advent"
#"reindeer walk gingerbread bake the night and sleep scrooge drive chimney jump elf laugh mistletoe give family advent fireplace ornament"
#reindeer jump walk gingerbread bake the night and sleep scrooge drive elf laugh mistletoe give family advent fireplace chimney ornament

# Batch-enabled scorer
* Re-enabled 8-bit quantization
* Make use of T4x2 setup
* Credit: https://www.kaggle.com/code/cdeotte/brute-force-first-sample-perplexity-470

In [5]:
class LRUCache:
    def __init__(self, capacity=10**11):
        self.capacity = capacity
        self.cache = OrderedDict()

    def get(self, key):
        if key in self.cache:
            self.cache.move_to_end(key)
            return self.cache[key]
        return None

    def set(self, key, value):
        self.cache[key] = value
        self.cache.move_to_end(key)
        if len(self.cache) > self.capacity:
            self.cache.popitem(last=False)

    def __len__(self):
        return len(self.cache)

In [6]:
import gc
import os
from math import exp
from collections import Counter
from typing import List, Optional, Union

import numpy as np
import pandas as pd
import transformers
import torch

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
PAD_TOKEN_LABEL_ID = torch.nn.CrossEntropyLoss().ignore_index
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class PerplexityCalculator:
    model_kwargs = {
        "attn_implementation": "sdpa",
        "device_map": "auto",
        "torch_dtype": torch.float16,
    }
    device = torch.device('cuda')

    def __init__(self,  model_path: str = "/kaggle/input/gemma-2/transformers/gemma-2-9b/2", capacity=10**11):
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_path, padding_side="right")
        self.model = transformers.AutoModelForCausalLM.from_pretrained(model_path, **self.model_kwargs)
        self.loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
        self.pad_token_label_id = self.loss_fct.ignore_index

        self.model.eval()
        self.cache = LRUCache(capacity=capacity)


    def get_perplexity(self, input_texts, batch_size=128, use_cache=True, verbose=False):
        single_input = isinstance(input_texts, str)
        input_texts = [input_texts] if single_input else input_texts
        
        results = [None] * len(input_texts)
        
        if use_cache:
            text_to_process = []
            for i, text in enumerate(input_texts):
                cached_val = self.cache.get(text)
                if cached_val is not None:
                    results[i] = cached_val
                else:
                    text_to_process.append(text)
        else:
            text_to_process = input_texts.copy()

        loss_list = []
        batches = len(text_to_process)//batch_size + (len(text_to_process)%batch_size != 0)
        pbar = range(batches)

        if verbose and batches:
            pbar = tqdm(range(batches))

        for j in pbar:

            a = j*batch_size
            b = (j+1)*batch_size
            input_batch = text_to_process[a:b]

            with torch.no_grad():

                # Explicitly add sequence boundary tokens to the text
                text_with_special = [f"{self.tokenizer.bos_token}{text}{self.tokenizer.eos_token}" for text in input_batch]

                # Tokenize
                model_inputs = self.tokenizer(
                    text_with_special,
                    return_tensors='pt',
                    add_special_tokens=False,
                    padding=True,
                )

                if 'token_type_ids' in model_inputs:
                    model_inputs.pop('token_type_ids')

                model_inputs = {k: v.to(self.device ) for k, v in model_inputs.items()}

                # Get model output
                output = self.model(**model_inputs, use_cache=False)
                logits = output['logits']

                label = model_inputs['input_ids']
                label[label == self.tokenizer.pad_token_id] = self.pad_token_label_id

                # Shift logits and labels for calculating loss
                shift_logits = logits[..., :-1, :].contiguous()  # Drop last prediction
                shift_labels = label[..., 1:].contiguous()  # Drop first input

                # Calculate token-wise loss
                loss = self.loss_fct(
                    shift_logits.view(-1, shift_logits.size(-1)),
                    shift_labels.view(-1)
                )

                loss = loss.view(len(logits), -1)
                valid_length = (shift_labels != self.pad_token_label_id).sum(dim=-1)
                loss = torch.sum(loss, -1) / valid_length

                loss_list += loss.cpu().tolist()

        ppl = [exp(i) for i in loss_list]

        index_ppl = 0
        for index_el, el in enumerate(results):
            if el is None:
                results[index_el] = ppl[index_ppl]
                self.cache.set(text_to_process[index_ppl], ppl[index_ppl])
                index_ppl += 1
        return results[0] if single_input else results

# Load custom scorer

In [7]:
#gemma-2-9b (competition scoring metric)
scorer = PerplexityCalculator('/kaggle/input/gemma-2/transformers/gemma-2-9b/2')

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

# Get our starting scores
* On initial dataset scorer returns some NaN's - we'll account for that..
* Since we've re-run this notebook a few times - further re-runs may not improve the score much...

In [8]:
%%time
# Get actual mean value 
scores = []
for row in range(len(samples)):
    score = scorer.get_perplexity(samples.iloc[row].text, batch_size=2, use_cache=True, verbose=False)
    print(samples.iloc[row].text)
    print(f"Score: {score:.2f}\n")
    scores.append(score)

print(f"Starting mean score: {np.mean(scores):.2f}")

reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament
Score: 467.99

reindeer sleep walk the night and drive mistletoe scrooge laugh gingerbread bake chimney elf jump ornament give family advent fireplace
Score: 439.63

sleigh yuletide beard carol cheer chimney decorations gifts grinch holiday holly jingle magi naughty nice nutcracker ornament polar workshop stocking
Score: 297.47

sleigh of the magi yuletide cheer is unwrap gifts relax and eat cheer decorations carol sing chimney visit workshop grinch holiday holly jingle naughty nice nutcracker polar beard ornament stocking
Score: 195.85

of and to in the as you that it we with from have not night season eggnog milk chocolate candy peppermint cookie fruitcake toy doll game puzzle greeting card wrapping paper bow wreath poinsettia star angel snowglobe candle fireplace wish dream believe wonder hope joy peace merry hohoho kaggle workshop
Score: 67.68

from and and as and have the in is it of not that the t

# Our "Simulated Annealing" Implementation

Key parameters for this optimizer:

1. Temperature progression:
   - Starts at `temp_start=3.0`, increases by `temp_start_increase_per_cycle=1.0` each cycle
   - Each cycle cools to `temp_end=0.2` using `cooling_rate=0.95`
   - Runs total of `reheat_cycles=8` cycles (so final cycle starts at 10.0)

2. Temperature step behavior:
   - Makes `steps_per_temp=170` attempts at each temperature level
   - This **results in a ~5 hour runtime** (assuming samples 0 and 5 are skipped)
   - When improvements found, temporarily drops to `temp_end` for cycles by `low_temp_samples_after_improve`
   - Returns to original temperature to continue cooling cycle

3. Batch processing efficiency:
   - Evaluates up to `max_batch_size=64` arrangements simultaneously
   - Limits total words processed to `max_words_per_batch=640` to manage memory
   - Automatically adjusts batch size based on sequence length

4. Between-cycle behavior:
   - Maintains best arrangement found across all cycles
   - Has `shuffle_chance_between_cycles=0.0` probability of randomizing sequence
   - Each new cycle starts at higher temperature to explore further from optimum

### Understanding Temperature and Acceptance Probabilities
Acceptance probability = exp(-delta/T), where delta is score difference and T is temperature

* At max temp (10.0): +1 worse: 90% chance, +5: 61%, +10: 37%, +20: 14%
* At starting temp (3.0): +1 worse: 72% chance, +5: 19%, +10: 4%, +20: 0.1%
* At min temp (0.2): +1 worse: 0.67% chance, +5 or worse: ≈0%

Better-scoring arrangements are always accepted. Higher temperatures allow exploring further from current solution, while minimum temperature mainly accepts improvements.
   
Progress indicators:
* `>` - found a new best score
* `<` - accepted a worse score (for exploration)
* `-` - rejected the new arrangement (only prints 10% of the time)
* `x` - skipped invalid arrangement (NaN score)

In [9]:
# ******************** mutate_sentence *********************
# Make a small change to a list of words
# Picks one of 5 possible mutations to the list
# - swap 2 words
# - remove 1 word and reinsert it at random
# - pick 4 indices and shuffle the words at those locations
# - remove a segment of words and reinsert it at random
# - same as above, but flip the segment
def mutate_sentence(sentence_in):
    sentence = sentence_in.copy()
    #print('       , , ',  sentence)
    n = len(sentence)
   
    if np.random.randint(100) < 80: # make a small mutation 
        if np.random.randint(100) < 33: # swap 2 words
            i, j = random.sample(range(n), 2)
            sentence[i], sentence[j] = sentence[j], sentence[i]
            #print('swap',i, j,  sentence)
            return(sentence)
        elif  np.random.randint(100) < 50:                           # shift 1 word to another spot
            i = np.random.choice(n) # word in slot i
            j = np.random.choice(n) # slot j
            t = sentence.pop(i)
            sentence.insert(j, t)
            #print('shift',i, j,  sentence)
            return(sentence)
        else:  # pick 4 random slots and permute the words in place
            t = 4
            p0 = random.sample(range(n), t)
            #print(p0)
            p1 = random.sample(p0, t)
            #print(p1)
            sentence[p0[0]], sentence[p0[1]], sentence[p0[2]], sentence[p0[3]] = sentence[p1[0]], sentence[p1[1]], sentence[p1[2]], sentence[p1[3]]
            if n != len(sentence): print('I fail at math')
            #print('       , , ',  sentence)
        return(sentence)
     # otherwise make a larger change - we'll move a segment
    i, j = random.sample(range(n + 1), 2)
    if i > j: i, j = j, i
   
    if np.random.randint(100) < 90:   # pick a random segment and relocate it
        segment = sentence[i:j]
        remains = sentence[:i]
        if j < n: remains = remains + sentence[j:]
        t = np.random.choice(len(remains)+ 1) # insert it here
        sentence = remains[:t] + segment + remains[t:]
        if n != len(sentence): print('I fail at math')
        #print('segment',i, j,  sentence)
        return(sentence)
    else:            # pick a random segment, reverse it and relocate that
        segment = sentence[i:j]
        segment.reverse()
        remains = sentence[:i]
        if j < n: remains = remains + sentence[j:]
        t = np.random.choice(len(remains)+ 1) # insert it here
        sentence = remains[:t] + segment + remains[t:]
        if n != len(sentence): print('mutate_sentence() has gone very wrong')
        #print('segment',i, j,  sentence)
        return(sentence)
    return(sentence)

This is the mutation function used for SA:

In [10]:
def mutate(text):
    child = text.copy()
    # 候補リスト
    mutation_types = ['swap2', 'swap3', 'swap', 'sort', 'sort2', 'shift', 'shiftlarge', 'inversion', 'scramble']
    
    # インデックスが若いほど大きい重みを付ける
    # 例: [9, 8, 7, 6, 5, 4, 3, 2, 1] のような降順
    n = len(mutation_types)
    weights = [1 for i in range(n)]  # 9,8,7,...,1
    
    # 重み付きで1つ選択する
    mutation_type = random.choices(mutation_types, weights=weights, k=1)[0]
    #child = child.split() # Convert child to a list of words 
    n = len(child) 
    if mutation_type == "swap2":
        i,j = random.sample(range(n),2)
        child[i], child[j] = child[j], child[i] 
    elif mutation_type == "swap3":
        i,j,k = random.sample(range(n),3)
        child[i], child[j], child[k] = child[j], child[k], child[i] 
    elif mutation_type == 'swap': 
        #mutation_size = random.randint(1, n//3) 
        mutation_size = random.randint(1, 4)
        for _ in range(mutation_size): 
            i, j = random.sample(range(n), 2) 
            child[i], child[j] = child[j], child[i] 
    elif mutation_type == "sort":
        segment_length = random.randint(3, n) 
        start = random.randint(0, n - segment_length) 
        end = start + segment_length
        child[start:end] = sorted(child[start:end])
    elif mutation_type == "sort2":
        segment_length = random.randint(3, n) 
        start = random.randint(0, n - segment_length) 
        end = start + segment_length
        child[start:end] = sorted(child[start:end])
    elif mutation_type == 'shift':
        i = np.random.choice(n) # word in slot i
        j = np.random.choice(n) # slot j
        t = child.pop(i)
        child.insert(j, t)
    elif mutation_type == 'shiftlarge':
        i = np.random.randint(0, n-1) # word in slot i 
        k = np.random.randint(1, 4) # length of the segment to move (1 to 4) 
        j = np.random.randint(0, n-1)
        # Ensure j is not within the segment to be moved 
        while j >= i and j <= i + k - 1: 
            j = np.random.randint(0, n-1)
        # Extract the segment and remove it from the list 
        t = child[i:i+k] 
        del child[i:i+k] 
        # Insert the segment at the new position 
        for index, word in enumerate(t): 
            child.insert(j + index, word)
    elif mutation_type == 'inversion': 
        segment_length = random.randint(3, 5) 
        start = random.randint(0, n - segment_length) 
        end = start + segment_length - 1
        child[start:end + 1] = reversed(child[start:end + 1])
    elif mutation_type == 'scramble': 
        segment_length = random.randint(3, 5) 
        start = random.randint(0, n - segment_length) 
        end = start + segment_length - 1 
        subset = child[start:end+1] 
        random.shuffle(subset) 
        child[start:end+1] = subset 
    elif mutation_type == "shuffle":
        random.shuffle(child)
    if text == child:
        child = mutate(child)
    return child #" ".join(child)

Change the parameters here:

In [11]:
from sortedcontainers import SortedSet, SortedList, SortedDict

In [12]:
math.log(0.1)/math.log(0.8)

10.318851158516171

In [13]:
# temp_start = 1 #30 #30 #50 #3.0                      # Initial temperature for first cycle
# temp_end = 0.1                        # Final/minimum temperature
# reheat_cycles = 2  #8                   # Number of full heating/cooling cycles to run
# temp_start_increase_per_cycle = 1.0 #1  # How much to increase starting temp each cycle
# cooling_rate = 0.80 #0.95                   # Temperature reduction multiplier each step 
# steps_per_temp = 300 #170 #300 #170                   # Attempts at each temperature level
# low_temp_samples_after_improve = steps_per_temp * 5  # Samples done at min temp after improvements (extends cycle)
# max_batch_size = 64           # Maximum sequences to evaluate at once
# max_words_per_batch = 640            # Total word limit per batch to manage memory
# shuffle_chance_between_cycles = 1.0   # Probability of completely shuffling the sequence between cycles
# max_len_text_que = 100
# initial_trials = 10

# # 空白文字で単語に分割
# def split_into_words(char_list):
#     words = ''.join(char_list).split(' ')  # リストを文字列に結合し、空白で分割
#     return words



# def simulated_annealing_optimize(text: str, pretext = "", temp_start=temp_start, temp_end=temp_end, 
#                                cooling_rate=cooling_rate, samples_per_temp=steps_per_temp, 
#                                max_batch_size=max_batch_size, max_words_per_batch=max_words_per_batch,
#                                reheat_cycles=reheat_cycles, low_temp_samples_after_improve=low_temp_samples_after_improve, shuffle_chance_between_cycles = shuffle_chance_between_cycles, verbose=False,
#                                 max_len_text_que = max_len_text_que, initial_trials=initial_trials):
#     text_que = SortedSet()
#     print(f"text: {text}")

#     words_list = text.split()
#     # text = f"'{text}'"
#     # print(f"text: {text}")
#     score = scorer.get_perplexity(text)
#     # print(f"words_list: {words_list}, joined_text: {' '.join(words_list)}")
#     text_que.add((score, tuple(words_list), temp_start))
#     best = ''
#     best_score = 1 << 60
#     print(f"これからSAでの探索を開始します.")
#     while len(text_que) > 0:
#         current_score, current_tuple, current_temp = text_que.pop()
#         current = list(current_tuple)  # リストに戻す
        
#         # print(f"current:{current}")
#         # 単語リスト
#         # current = split_into_words(current)
#         current_temp *= cooling_rate
#         print(f"current_score: {current_score}, current: {' '.join(current)}")
        
#         neighbors = []
#         neighbor_texts = []
#         for _ in range(max_batch_size):
#             neighbor = current.copy()
#             neighbor = mutate(neighbor)
#             neighbors.append(neighbor)
#             neighbor_texts.append(pretext + ' '.join(neighbor))
#             # print(f"neighbor_text: {' '.join(neighbor)}")
        
#         neighbor_scores = scorer.get_perplexity(neighbor_texts, batch_size=max_batch_size, use_cache=True, verbose=False)
#         for neighbor, neighbor_score in zip(neighbors, neighbor_scores):
#             if math.isnan(neighbor_score):
#                 if verbose:
#                     print("x", end="", flush=True)
#                 continue
#             if best_score * 1.5 < neighbor_score:
#                 continue
#             delta = neighbor_score - current_score
#             # if delta < 0 or random.random() < math.exp(-delta / current_temp):
#             # 現在はランダム性が全くないので改善できそう
            
#             if delta < 0 or random.random() < math.exp(-delta / current_temp):
#                 # print(f"add_neighbor: {neighbor}")
#                 text_que.add((neighbor_score, tuple(neighbor), current_temp))
#                 if best_score > neighbor_score:
#                     best_score = neighbor_score
#                     best = neighbor
#                     print(f"\n best_score: {best_score}, {' '.join(best)}")
#         print(f"len_text_que: {len(text_que)}, best_score: {best_score}")
#         while len(text_que) > max_len_text_que:
#             text_que.pop()

#     print(f"best: {best}")
    
#     return pretext + ' '.join(best), best_score


In [14]:
for idx, row in samples.iterrows():
    print(idx, row.text)

0 reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament
1 reindeer sleep walk the night and drive mistletoe scrooge laugh gingerbread bake chimney elf jump ornament give family advent fireplace
2 sleigh yuletide beard carol cheer chimney decorations gifts grinch holiday holly jingle magi naughty nice nutcracker ornament polar workshop stocking
3 sleigh of the magi yuletide cheer is unwrap gifts relax and eat cheer decorations carol sing chimney visit workshop grinch holiday holly jingle naughty nice nutcracker polar beard ornament stocking
4 of and to in the as you that it we with from have not night season eggnog milk chocolate candy peppermint cookie fruitcake toy doll game puzzle greeting card wrapping paper bow wreath poinsettia star angel snowglobe candle fireplace wish dream believe wonder hope joy peace merry hohoho kaggle workshop
5 from and and as and have the in is it of not that the to we with you yuletide cheer advent angel bake beard believe b

# Run and Submit!

Change the text to optimize here: either a full text, then use split afterwards (processing_words.split()). Or define the words, combining words which should processed together. A pretext or aftertext can also be added, it has to be added within the code (see #!!!!!!!!!!!!!!!!) and do not forget the empty space. This text has to be removed from the words and will be kept unchanged at the beginning (or end).

In [15]:
text = "from and and as and have the in is it of not that the to we with you yuletide cheer advent angel bake beard believe bow candy candle carol cheer chocolate chimney cookie decorations doll dream drive eat eggnog elf family fireplace fireplace chimney fruitcake game give gifts gingerbread grinch greeting card holly hohoho holiday hope jingle jump joy kaggle laugh magi merry milk mistletoe naughty nice night nutcracker ornament ornament of the night peace peppermint polar poinsettia puzzle reindeer relax scrooge season sing sleigh sleep snowglobe star stocking toy unwrap visit walk wish wonder workshop workshop wreath wrapping paper"

scorer.get_perplexity(text, batch_size=1, use_cache=False, verbose=False)

33.31005738633129

In [16]:
import optuna

# テキストを単語に分割
text = "sleigh of the magi yuletide cheer is unwrap gifts relax and eat cheer decorations carol sing chimney visit workshop grinch holiday holly jingle naughty nice nutcracker polar beard ornament stocking"
words = text.split()

# 分割数
num_splits = 8
num_split_points = num_splits - 1
total_words = len(words)

# パープレキシティを計算する関数（仮）
# 実際の scorer.get_perplexity に置き換えてください
def get_perplexity(sub_text):
    # ここではダミーのパープレキシティを返します。
    # 実際には scorer.get_perplexity を使用してください。
    return scorer.get_perplexity(sub_text, batch_size=1, use_cache=False, verbose=False)

# 目的関数の定義
def objective(trial):
    split_points = []
    lower = 1
    for i in range(num_split_points):
        # 残りの分割点に必要な最低位置を確保
        max_split = total_words - (num_split_points - i)
        split = trial.suggest_int(f'split_{i}', lower, max_split)
        split_points.append(split)
        lower = split + 1  # 次の分割点は現在の分割点より後に設定

    split_points = sorted(split_points)

    # 分割点に基づいてテキストを分割
    splits = []
    previous = 0
    for point in split_points:
        split = ' '.join(words[previous:point])
        splits.append(split)
        previous = point
    # 最後の部分
    splits.append(' '.join(words[previous:]))

    # パープレキシティの合計を計算
    total_perplexity = 0
    for split in splits:
        perplexity = get_perplexity(split)
        total_perplexity += perplexity

    return total_perplexity

# Optunaスタディの作成
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# 最適な分割点の取得
best_split_points = sorted([study.best_params[f'split_{i}'] for i in range(num_split_points)])
splits = []
previous = 0
for point in best_split_points:
    split = ' '.join(words[previous:point])
    splits.append(split)
    previous = point
splits.append(' '.join(words[previous:]))

# 結果の表示
print("最適な分割点:", best_split_points)
print("分割後のテキスト:")
for i, split in enumerate(splits, 1):
    print(f"Part {i}: {split}, score: {get_perplexity(split)}")

print("合計パープレキシティ:", study.best_value)



[I 2024-12-30 14:38:07,330] A new study created in memory with name: no-name-4209a898-d5c0-44ff-87db-6f15fa7a2278
[I 2024-12-30 14:38:08,145] Trial 0 finished with value: 68314921.32550278 and parameters: {'split_0': 13, 'split_1': 16, 'split_2': 17, 'split_3': 19, 'split_4': 25, 'split_5': 27, 'split_6': 28}. Best is trial 0 with value: 68314921.32550278.
[I 2024-12-30 14:38:08,923] Trial 1 finished with value: 446926571.69260687 and parameters: {'split_0': 7, 'split_1': 19, 'split_2': 22, 'split_3': 26, 'split_4': 27, 'split_5': 28, 'split_6': 29}. Best is trial 0 with value: 68314921.32550278.
[I 2024-12-30 14:38:09,697] Trial 2 finished with value: 7487206.266702815 and parameters: {'split_0': 1, 'split_1': 4, 'split_2': 11, 'split_3': 17, 'split_4': 25, 'split_5': 26, 'split_6': 29}. Best is trial 2 with value: 7487206.266702815.
[I 2024-12-30 14:38:10,490] Trial 3 finished with value: 481452060.22252893 and parameters: {'split_0': 22, 'split_1': 24, 'split_2': 25, 'split_3': 26, 

最適な分割点: [1, 3, 5, 13, 22, 26, 29]
分割後のテキスト:
Part 1: sleigh, score: 107574.34834416473
Part 2: of the, score: 63735.692088290736
Part 3: magi yuletide, score: 2358.13186045924
Part 4: cheer is unwrap gifts relax and eat cheer, score: 9039.643348424153
Part 5: decorations carol sing chimney visit workshop grinch holiday holly, score: 21855.054478082897
Part 6: jingle naughty nice nutcracker, score: 19899.25355081959
Part 7: polar beard ornament, score: 3047059.2404478486
Part 8: stocking, score: 258057.44650742714
合計パープレキシティ: 3529578.810625517


In [ ]:
import itertools
from tqdm import tqdm  # プログレスバーの表示に使用

# すべての順列を生成
all_permutations = itertools.permutations(splits)

# 最小パープレキシティスコアと対応する順列を保存する変数
min_perplexity = float('inf')
best_permutation = None

# 全順列に対してパープレキシティを計算
for perm in tqdm(all_permutations, total=40320, desc="Evaluating permutations"):
    # 順列を結合して一つのテキストにする
    permuted_text = ' '.join(perm)
    
    # パープレキシティを計算
    perplexity = get_perplexity(permuted_text)
    
    # 最小パープレキシティスコアの更新
    if perplexity < min_perplexity:
        min_perplexity = perplexity
        best_permutation = perm

# 結果の表示
print("\n最適な順列のパープレキシティ:", min_perplexity)
print("最適な順列:")
for i, part in enumerate(best_permutation, 1):
    print(f"Part {i}: {part}")


Evaluating permutations:  12%|█▏        | 4886/40320 [09:42<1:10:03,  8.43it/s]

In [ ]:
submission = pd.DataFrame(columns=['id', 'text'])
scores = []
texts = []
# processing_words = ['magi yuletide cheer', 'grinch', 'carol', 'holly', 'jingle', 'naughty nice', 
#                     'nutcracker', 'polar', 'beard', 'ornament', 'stocking chimney', 'sleigh', 
#                     'holiday', 'workshop',  
#                     'gifts', 'decorations']
# processing_words = ['sleigh','naughty nice', 'grinch','gifts','magi','yuletide', 'workshop', 'nutcracker',
#                     'holiday','holly jingle', 'carol', 
#                     'stocking', 'chimney', 'polar', 'beard', 'ornament',  
#                     'decorations']
# processing_words = ['sleigh of the magi', 'is', 'yuletide', 'cheer', 'unwrap gifts','relax', 
#                     'and', 'eat', 'holiday', 'decorations', 'holly jingle',
#                     'carol','cheer', 'sing', 'chimney', 'visit', 'workshop',
#                     'grinch naughty nice polar beard nutcracker ornament stocking']
#processing_words = ["from", "and", "as", "have", "in", "not", "it", "of", "that", "the", "to", "we", "with", "you", 
#"season greeting angel believe bow card candle candy chocolate cookie doll dream eggnog fireplace fruitcake game hohoho hope joy kaggle merry milk night peace peppermint poinsettia puzzle snowglobe star toy wreath wish workshop wonder wrapping paper"]
#processing_words = ['wreath', 'it', 'have', 'merry', 'season', 'hohoho', 'and to you', 'from the star', 'of', 'wonder', 'workshop', 'that night', 'not with', 'joy', 
#                    'as we', 'believe in', 'hope', 'peace', 'milk', 'cookie', 'eggnog', 'fruitcake', 'chocolate', 'candy', 'peppermint', 'candle', 'snowglobe', 
#                    'angel', 'poinsettia', 'wrapping paper', 'bow', 'greeting card', 'wish', 'dream', 'fireplace', 'kaggle', 'toy', 'doll', 'game', 'puzzle']
# processing_words = [#"from and as have in not it of that the to we with you season",
#                      'joy', 'wonder', 'dream', 'believe', 'wish', 'hope', 'peace', 'star', 'night',
#                     'candle', 'wreath', 'angel', 'snowglobe', 'fireplace', 'poinsettia', 'candy',
#                     'peppermint', 'chocolate', 'cookie', 'milk', 'eggnog', 'fruitcake',
#                     "greeting card wrapping paper bow toy doll game puzzle merry hohoho kaggle workshop"]
# processing_words = ['of', 'and', 'to', 'and', 'in', 'the', 'as', 'and', 'is', 
#                     'that', 'the', 'it', 'we', 'with', 'from', 'have', 'not', 
#                     'you yuletide cheer advent angel bake beard believe bow candy candle carol cheer chocolate chimney cookie decorations doll dream drive', 
#                     'eat eggnog elf family fireplace fireplace chimney fruitcake game give gifts gingerbread grinch greeting card', 
#                     'holly hohoho holiday hope jingle jump joy kaggle laugh magi merry milk mistletoe naughty nice night nutcracker', 'ornament ornament', 'of', 'the', 
#                     'night peace peppermint polar poinsettia puzzle reindeer relax scrooge season sing sleigh sleep snowglobe star stocking toy unwrap visit walk wish wonder workshop workshop wreath wrapping paper'
#                     ]
# processing_words = "and and as have the in is it of the to we with you yuletide cheer advent and angel bake beard believe bow candy candle carol cheer chocolate chimney cookie decorations doll dream drive eat eggnog elf from family fireplace fireplace chimney fruitcake game give gifts gingerbread grinch greeting card holly hohoho holiday hope jingle jump joy kaggle laugh magi merry milk mistletoe naughty nice nutcracker not that night ornament ornament of the night peace peppermint polar poinsettia puzzle reindeer relax scrooge season sing sleigh sleep snowglobe star stocking toy unwrap visit walk wish wonder workshop workshop wreath wrapping paper"
# processing_words = "and is of the sleigh yuletide beard cheer chimney decorations grinch holiday holly jingle magi naughty nice nutcracker ornament polar stocking workshop cheer carol eat sing relax visit unwrap gifts "
# processing_words = "sleigh of the magi yuletide cheer is eat relax and unwrap gifts cheer grinch holiday holly jingle naughty nice nutcracker ornament polar beard workshop visit chimney stocking carol sing decorations"
# processing_words = ["sleigh of the magi yuletide cheer is", 'eat relax and unwrap gifts', 'cheer', 'grinch holiday holly jingle naughty nice nutcracker', 'ornament', 'polar', 'beard', 'visit workshop', 'chimney', 'stocking', 'carol sing', "decorations"]
# processing_words = ['of', 'magi', 'yuletide cheer', 'and', 'unwrap', 'gifts', 'sing', 'carol','the','holly',  'jingle', 'eat', 'holiday', 'visit workshop', 'grinch', 'is', 'naughty', 
#                     'nice','relax','stocking', 'chimney', 'decorations', 'ornament', 'cheer', 'nutcracker', 'polar', 'beard']
#processing_words = "season joy wonder dream believe in wish and to you hope peace from the star of that night candle wreath angel snowglobe fireplace poinsettia candy peppermint chocolate cookie milk not eggnog fruitcake greeting card wrapping paper bow it toy doll game puzzle have merry hohoho as we with kaggle workshop"
#processing_words = "magi yuletide grinch holiday cheer gifts decorations ornament stocking holly jingle sleigh carol naughty nice polar beard workshop chimney nutcracker"
#processing_words = "sleigh of the magi yuletide cheer unwrap gifts relax and eat cheer carol sing holiday decorations holly jingle workshop visit grinch naughty is nice nutcracker ornament stocking chimney polar beard"
processing_words = "grinch carol magi yuletide cheer holiday holly jingle naughty nice nutcracker polar beard ornament stocking chimney sleigh workshop gifts decorations"
#processing_words = processing_words.split()
#processing_words = samples.loc[1,"text"].split()
#processing_words = processing_words.split()
#"sleigh magi yuletide cheer gifts holiday decorations holly jingle carol chimney grinch naughty nice polar beard workshop nutcracker ornament stocking"

# Process each sample
id_texts = {
    3: 'sleigh yuletide is unwrap gifts of cheer eat jingle relax sing carol the magi visit workshop naughty and nice holiday holly cheer chimney grinch nutcracker polar beard ornament decorations stocking',
    # 必要に応じて、他の id 用のテキストを追加できる
    # たとえば 4: "other text ..."
}
texts_id3 = [
    "sleigh yuletide beard carol cheer cheer chimney decorations eat gifts grinch holiday holly jingle is naughty and nice nutcracker ornament of the magi polar relax sing stocking unwrap visit workshop", 
    "sleigh and cheer cheer beard chimney carol decorations eat gifts grinch holiday holly jingle is naughty nice nutcracker ornament of the magi polar relax sing stocking unwrap visit workshop yuletide"
    ]
        

for idx, row in samples.iterrows():

    if idx in skip:
        score = scorer.get_perplexity(row.text, batch_size = 1)
        print(f"Skipping sample {idx} ({score:.2f})")
        final_text = row.text
    else:
        print(f"\nProcessing sample {idx}...")
        text = samples.iloc[idx].text
        print(f"text: {text}")
        words_list = text.split()
        fixed_word = words_list[0]
        remaining_words = words_list[1:]
        random.shuffle(remaining_words)
        shuffled_words = [fixed_word] + remaining_words
        shuffled_text = ' '.join(shuffled_words)
        print(f"shuffled_text: {shuffled_text}")
        
        # 最適化関数の呼び出し
        simulated_annealing_optimize(shuffled_text, verbose=True)
        
        print("正常に終了しました。")
        
        
        
    
    scores.append(score)
   
    # Add to submission dataframe
    #submission.loc[idx] = {
    #    'id': row.id,
     #   'text': final_text
    #}
    #print("-" * 80)

# Print summary statistics
print("\nScore Summary:")

for i in range(len(scores)):
   print(f"Sample {i} final score: {scores[i]:.2f}")

print(f"Submission mean score: {np.mean(scores):.2f}")

# Save to CSV
#submission.to_csv("submission.csv", index=False)
#print("\nSubmission file created!")

#submission

課題としては
思うようにスコアが減少しない。
バッチサイズの問題なのかスタートの温度の問題なのか。
最初の文字列に依存して得られるスコアが大きく変動しそう。


以下の大きなブラックホールに吸い込まれてしまっている、どう脱出するか。もしくは知被かないかを検討する必要がある。

-New best score: 223.66653284614108


New best text: sleigh yuletide beard carol cheer cheer chimney decorations eat gifts grinch holiday holly jingle is naughty and nice nutcracker ornament of the magi polar relax sing stocking unwrap visit workshop

文章内での複数単語を一単語として計算するとよいスコアが出る可能性がある。そこでどの単語を連語として計算するかを考える必要がある。

In [ ]:
import torch

# デフォルトデバイスをCPUに設定
device = torch.device('cpu')
